Deactivating callbacks can be especially useful when you do a parameter search (say with sklearn GridSearchCV). If, for instance, you use a callback for learning rate scheduling (e.g. via LRScheduler) and want to test its usefulness, you can compare the performance once with and once without the callback.

implement gridSearch

implement space_cnn

implement long core for cnn


# Choose Signal and Import Data

In [1]:
import os
import numpy as np
import scipy.io

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
mods = ['BPSK', 'DQPSK', 'GFSK', 'GMSK', 'OQPSK',
        'PAM4', 'PAM8', 'PSK8', 'QAM16', 'QAM64', 'QPSK']
class_num = len(mods)

In [4]:
data = scipy.io.loadmat(
    "D:/batch100000_symbols128_sps8_baud1_snr5.dat",
)

In [5]:
def import_from_mat(data, size):
    features = []
    labels = []
    for mod in mods:
        real = np.array(data[mod].real[:size])
        imag = np.array(data[mod].imag[:size])
        signal = np.concatenate([real, imag], axis=1)
        features.append(signal)
        labels.append(mods.index(mod) * np.ones([size, 1]))

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [6]:
features, labels = import_from_mat(data,100000)

In [7]:
features = features.astype(np.float32)
labels = labels.astype(np.int64)

In [8]:
X = features
y = labels.reshape(-1)

# Define Model

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class Discriminator(nn.Module):
    """Define the model"""

    def __init__(self, dr):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(2, 256, 3, padding=1),  # batch, 256, 1024
            nn.BatchNorm1d(256),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 80, 3, padding=1),  # batch, 80, 1024
            nn.BatchNorm1d(80),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(80 * 1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=dr)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, class_num),
            nn.ReLU()
        )
        
    def forward(self, x, **kwargs):
        x = x.reshape((x.size(0), 2, -1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

# Define Classifier and Callback. 
The Callbacks are used to calculate score and print train process

In [11]:
from skorch import NeuralNetClassifier
from skorch.callbacks import Callback, EpochScoring, Checkpoint, EarlyStopping
from sklearn.metrics import confusion_matrix
from skorch.utils import data_from_dataset

In [12]:
class Score_ConfusionMatrix(EpochScoring):
    def on_epoch_end(self, net, dataset_train, dataset_valid, **kwargs):
        
        EpochScoring.on_epoch_end(self, net, dataset_train, dataset_valid)
        
        X_test, y_test = data_from_dataset(dataset_valid)
        y_pred = net.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        history = net.history
        history.record("confusion_matrix", cm)

In [13]:
class Print_Score_CM(Callback):
    
    def __init__(self, size):
        self.batch_num = 0
        self.sample_num = 0
        self.size = size
        
    def on_batch_end(self, net, **kwargs):
        self.batch_num += 1
        self.sample_num += kwargs['X'].shape[0]
        if self.batch_num % 1 == 0:
            percent = self.sample_num/self.size
            history = net.history
            training = kwargs["training"]
            if training:
                print('processed: {0:.4f}, ' 
                      'train_batch: {1}, '
                      'train_loss:{2:.4f}'.format(
                          percent,
                          history[-1, "batches", -1, "train_batch_size"],
                          history[-1, "batches", -1, "train_loss"]
                      ))
            else:
                print('processed: {0:.4f}, ' 
                      'valid_batch: {1}, '
                      'valid_loss:{2:.4f}'.format(
                          percent,
                          history[-1, "batches", -1, "valid_batch_size"],
                          history[-1, "batches", -1, "valid_loss"]
                      ))
    def on_epoch_end(self, net, **kwargs):
        self.batch_num = 0
        self.sample_num = 0
        history = net.history
        result = history[-1].copy()
        result.pop("confusion_matrix")
        result.pop("batches")
        print("epoch: {0}, "
              "dur: {1:.2f}s, "
              "val_acc: {2:.4f}{3}, "
              "val_loss: {4:.4f}{5}, "
              "saved: {6}".format(
                  history[-1, "epoch"],
                  history[-1, "dur"],
                  history[-1, "accuracy"],
                  "(best)" if history[-1, "accuracy_best"] else "",
                  history[-1, "valid_loss"],
                  "(best)" if history[-1, "valid_loss_best"] else "",
                  history[-1, "event_cp"]
              ))
        print("Confusion matrix:\n {0}".format(
            history[-1, "confusion_matrix"]
        ))

In [14]:
cp = Checkpoint(dirname='best')
early_stop = EarlyStopping(patience=20)
net = NeuralNetClassifier(
    Discriminator,
    max_epochs=200,
    lr=0.01,
    module__dr=0.6,
    device='cuda',
    callbacks=[('best',cp),
               ('early', early_stop)
              ],
    iterator_train__shuffle=True,
    iterator_valid__shuffle=False
)

# score = Score_ConfusionMatrix(scoring="accuracy", lower_is_better=False)
# pt = Print_Score_CM(X.shape[0])

# net.set_params(callbacks__valid_acc=score)
# net.set_params(callbacks__print_log=pt)


# Train Classifier

In [15]:
net.fit(X,y)

  epoch    train_loss    valid_acc    valid_loss    cp       dur
-------  ------------  -----------  ------------  ----  --------
      1        0.3432       0.8901        0.2117     +  320.2734
      2        0.1563       0.9635        0.1007     +  318.2180
      3        0.1161       0.9695        0.0824     +  318.1572


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Discriminator(
    (conv1): Sequential(
      (0): Conv1d(2, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1d(256, 80, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (fc1): Sequential(
      (0): Linear(in_features=81920, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.6)
    )
    (fc2): Sequential(
      (0): Linear(in_features=256, out_features=11, bias=True)
      (1): ReLU()
    )
  ),
)

# Output confusion matrix

In [16]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [19]:
y_pred = net.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

In [20]:
print(cm)

[[30076     0     0     0     0     0     0     0     0     0     0]
 [    0 28374     0     0     1     0     0  1279     0     0   630]
 [    0     0 29880     0     0     0     0     0     0     0     0]
 [    0     0     0 29946     0     0     0     0     0     0     0]
 [    0     0     0     0 29835     0     0     2     0     0   100]
 [    0     0     0     0     0 29872     0     0     0     0     0]
 [    0     0     0     0     0     0 29998     0     0     0     0]
 [    0  1692     0     0    13     0     0 24179     0     0  4307]
 [    0     0     0     0     0     0     0     0 29800     0     0]
 [    0     0     0     0     0     0     0     0     0 29949     0]
 [    0   185     0     0    39     0     0   560     0     0 29283]]


# GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import norm

GridSearch (choose parameter to try, lr, warm_start_param（yes or not, parameters）, LRScheduler, dropout, max_epoch=200, patenice=20 )
对数尺度

In [ ]:
param_dist = {
    'lr': [0.0005, 0.001, 0.005, 0.01, 0.05],
    'module__dr': norm(loc=0.5, scale=0.1),
}

In [ ]:
random_search = RandomizedSearchCV(
    net,
    scoring='accuracy',
    param_distributions =param_dist,
    n_iter=20,
    cv = 5
)

In [ ]:
random_search.fit(X, y)